## 1. Importación de las bibliotecas necesarias

In [1]:
import glob as glob
import numpy as np
import cv2
import datetime
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras import Model 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.optimizers import SGD
from scipy.interpolate import make_interp_spline
import pandas as pd
import matplotlib.pyplot as plt

## 2. Lectura y preprocesamiento de datos (generador de imágenes)

### 2.1. Mostrar algunas imágenes de entrenamiento

In [ ]:
datagen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest')

#### - Fried Noodles

In [ ]:
img = load_img('data/train/noodles_fried/3685231_a86b9a9ec2_b.jpg')
plt.figure(figsize = (4,4))
plt.imshow(img)
plt.axis('off')
plt.title('Fried Noodles')
plt.show()

In [ ]:
x = img_to_array(img)
x2 = x.reshape((1,) + x.shape)

In [ ]:
i = 0
fig = plt.figure(figsize = (10,10))
plt.title('Pre-processed')
for batch in datagen.flow(x2, batch_size = 1):
    i += 1
    if i > 9:
        break
    temp = batch.reshape(x.shape)
    plt.subplot(3, 3, i)
    plt.imshow(temp.astype('uint8'))
    plt.axis('off')
plt.show()

#### - Noodle soup

In [ ]:
img = load_img('data/train/noodles_soup/5153183161_eb845fb8d9_b.jpg')
plt.figure(figsize = (4,4))
plt.imshow(img)
plt.axis('off')
plt.title('Noodle Soup')
plt.show()

In [ ]:
x = img_to_array(img)
x2 = x.reshape((1,) + x.shape)

In [ ]:
i = 0
fig = plt.figure(figsize = (10,10))
plt.title('Pre-processed')
for batch in datagen.flow(x2, batch_size = 1):
    i += 1
    if i > 9:
        break
    temp = batch.reshape(x.shape)
    plt.subplot(3, 3, i)
    plt.imshow(temp.astype('uint8'))
    plt.axis('off')
plt.show()

### 2.2. Lectura de datos de entrenamiento y validación

In [ ]:
# configuración por defecto
img_width, img_height = 299, 299

train_dir = 'data/train'
validate_dir = 'data/validate'
batch_size = 32
nb_classes = len(glob.glob(train_dir + '/*'))

### 2.3. Preprocesamiento de las imágenes

In [ ]:
# pre-procesamiento de datos para entrenamiento
train_datagen =  ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest',
    horizontal_flip = True)

In [ ]:
# pre-procesamiento de datos para su validación
validate_datagen =  ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest',
    horizontal_flip = True)

In [ ]:
# generar y almacenar datos de entrenamiento
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size)

In [ ]:
# generar y almacenar datos de validación
validate_generator = validate_datagen.flow_from_directory(
    validate_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size)

## 3. Construcción del modelo

### 3.1. Configurar la transferencia del conocimiento

In [ ]:
# Configure el aprendizaje de transferencia en el modelo ImageNet VGG19 previamente entrenado: 
# elimine la capa completamente conectada y reemplace con softmax para clasificar 2 clases
vgg19_model = VGG19(weights = 'imagenet', include_top = False)
x = vgg19_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(nb_classes, activation = 'softmax')(x)
model = Model(vgg19_model.input, predictions)

In [ ]:
# congelar todas las capas del modelo previamente entrenado
for layer in vgg19_model.layers:
    layer.trainable = False

### 3.2. Compilar el nuevo modelo

In [ ]:
# compile el nuevo modelo usando un optimizador RMSProp
model.compile(optimizer = 'rmsprop',
    loss = 'categorical_crossentropy',
    metrics = ['accuracy'])

In [ ]:
model.summary()

## 4. Entrenar el nuevo modelo mostrando el tiempo total de entrenamiento

In [ ]:
# ajustar el modelo, registrar los resultados y el tiempo de entrenamiento
now = datetime.datetime.now
t = now()
transfer_learning_history = model.fit(
    train_generator,
    epochs = 20,
    steps_per_epoch = len(train_generator),
    validation_data = validate_generator,
    validation_steps = len(validate_generator), verbose = 1)
print('Tiempo de entrenamiento: %s' % (now() - t))

## 5. Evaluar el desempeño del modelo

In [ ]:
# Evaluar el rendimiento del nuevo modelo e informar de los resultados
score = model.evaluate_generator(validate_generator, len(validate_generator))
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

## 6. Guardar el modelo de aprendizaje por transferencia para fines de predicción fuera de línea

In [ ]:
model.save('model/noodles_vgg19_model_tl.h5')

## 7. Gráficas de entrenamiento y validación

In [ ]:
xfer_acc = transfer_learning_history.history['accuracy']
val_acc = transfer_learning_history.history['val_accuracy']
xfer_loss = transfer_learning_history.history['loss']
val_loss = transfer_learning_history.history['val_loss']
epochs = range(len(xfer_acc))

x = np.array(epochs)
y = np.array(xfer_acc)
x_smooth = np.linspace(x.min(), x.max(), 500)
y_smooth = make_interp_spline(x,y)(x_smooth)
plt.plot(x_smooth, y_smooth, 'r-', label = 'Entrenamiento')

x1 = np.array(epochs)
y1 = np.array(val_acc)
x1_smooth = np.linspace(x1.min(), x1.max(), 500)
y1_smooth = make_interp_spline(x1,y1)(x1_smooth)

plt.plot(x1_smooth, y1_smooth, 'g-', label = 'Validación')
plt.title('Transfer Learning - Precisión de entrenamiento y validación')
plt.legend(loc = 'lower left', fontsize = 9)
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.ylim(0,1.05)

plt.figure()
x = np.array(epochs)
y = np.array(xfer_loss)
x_smooth = np.linspace(x.min(), x.max(), 500)
y_smooth = make_interp_spline(x,y)(x_smooth)
plt.plot(x_smooth, y_smooth, 'r-', label = 'Entrenamiento')

x1 = np.array(epochs)
y1 = np.array(val_loss)
x1_smooth = np.linspace(x1.min(), x1.max(), 500)
y1_smooth = make_interp_spline(x1,y1)(x1_smooth)

plt.plot(x1_smooth, y1_smooth, 'g-', label = 'Validación')
plt.title('Transfer Learning - Pérdida de entrenamiento y validación')
plt.legend(loc = 'upper right', fontsize = 9)
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.ylim(0,max(y1))
plt.show()

## 8. Validar el modelo

In [ ]:
predict_files = glob.glob("./data/test/*.jpg")

In [ ]:
im = cv2.imread(predict_files[0])
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
im = cv2.resize(im, (256, 256)).astype(np.float32)
im = np.expand_dims(im, axis = 0)/255

In [ ]:
predictor, image_id = [], []
for i in predict_files:
    im = cv2.imread(i)
    im = cv2.resize(cv2.cvtColor(im, cv2.COLOR_BGR2RGB), (256, 256)).astype(np.float32) / 255.0
    im = np.expand_dims(im, axis =0)
    outcome = [np.argmax(model.predict(im))]
    predictor.extend(list(outcome))
    image_id.extend([i.rsplit("\\")[-1]])

In [ ]:
final = pd.DataFrame()
final["id"] = image_id
final["Noodles"] = predictor
final.head(29)

In [ ]:
classes = train_generator.class_indices
classes = {value : key for key, value in classes.items()}

In [ ]:
final["Noodles"] = final["Noodles"].apply(lambda x: classes[x])
final.head(29)

**Grafcar** para poder ver todo el set de la predicción de las 2 clases.

In [ ]:
import numpy as np
import os
from keras.preprocessing.image import load_img, img_to_array
longitud, altura = 150, 150
clases = ['noodles_fried','noodle_soup']
path =  "./data/test"
Ima = os.listdir(path)
for w in Ima:
    RutaImg=path+'/'+w
    img = load_img(RutaImg, target_size=(longitud, altura))
    x = img_to_array(img)
    x = x / 255.0
    x = np.expand_dims(x, axis=0)
    array = model.predict(x)
    plt.figure(figsize=(6,3))
    plt.subplot(1,2,1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)
    predicted_label = np.argmax(array[0])
    if (w.find(clases[0]) != -1):
        i = 0
    elif (w.find(clases[1]) != -1):
        i = 1 

    if (predicted_label == i):
        color = 'blue'
    else:
        color = 'red'
    
    plt.xlabel("{} {:2.0f}% ({})".format(clases[predicted_label],100*np.max(array[0]),clases[i]),color=color)

    plt.subplot(1,2,2)
    plt.grid(False)
    plt.xticks(range(2))
    plt.yticks([])
    thisplot = plt.bar(range(2), array[0],color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(array[0])
    thisplot[predicted_label].set_color('red')
    thisplot[i].set_color('blue')
    plt.show()